# Projeto: Rede Neural Artificial para Detecção de Fraudes

## 1) Introdução

Este notebook demonstra o desenvolvimento de uma Rede Neural Artificial com dois focos:

1. **CNN em CIFAR-10**: exemplo clássico de boas práticas em visão computacional.
2. **MLP em fraude tabular**: problema real de detecção de fraude, caracterizado por dados desbalanceados (~1% positivos) e custo assimétrico.

A arquitetura e as justificativas seguem literatura especializada e boas práticas (Goodfellow et al., 2016; Srivastava et al., 2014; He et al., 2016).

## 2) CNN em CIFAR-10

Nesta seção treinamos uma CNN VGG-like com BatchNorm, Dropout, L2, GlobalAveragePooling e augmentation leve.


### 2.1 Carregamento e Pré-processamento dos Dados

Carregamos o **dataset CIFAR-10**, composto por 60 mil imagens coloridas (32×32, 3 canais) distribuídas em 10 classes.  
- Divisão: 50 mil imagens para treino e 10 mil para teste.  
- As labels são convertidas para **one-hot encoding** com `to_categorical`.  
- As imagens são **normalizadas** para o intervalo [0,1], facilitando a convergência da rede.  
- Por fim, conferimos os formatos de entrada e saída resultantes.


In [10]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(num_classes, input_shape) = (10, (32, 32, 3))
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train, y_test = to_categorical(y_train, num_classes), to_categorical(y_test, num_classes)
x_train, x_test = x_train/255.0, x_test/255.0
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 10))

### 2.2 Definição da Arquitetura da CNN

Aqui implementamos uma **CNN do tipo VGG-like** para o CIFAR-10, seguindo boas práticas:  
- **Camadas convolucionais** com kernel 3×3 e ativação ReLU para extração de padrões locais.  
- **Batch Normalization** após cada convolução para estabilizar o treino e acelerar a convergência.  
- **MaxPooling (2×2)** para reduzir dimensionalidade e capturar características hierárquicas.  
- **Dropout progressivo (0.25 → 0.45 → 0.5)** para mitigar overfitting.  
- **Regularização L2 (1e-4)** aplicada aos pesos.  
- **GlobalAveragePooling2D** em vez de Flatten, reduzindo número de parâmetros.  
- **Camada densa final (128 neurônios, ReLU)** seguida da saída softmax (10 classes).  

O modelo é compilado com **Adam (lr=1e-3)** e perda de **categorical crossentropy**, monitorando **acurácia**.

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

def make_cnn(input_shape, num_classes):
    reg = regularizers.l2(1e-4)
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3,3), padding="same", activation="relu", kernel_regularizer=reg)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3,3), padding="same", activation="relu", kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Conv2D(64, (3,3), padding="same", activation="relu", kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3,3), padding="same", activation="relu", kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Dropout(0.35)(x)

    x = layers.Conv2D(128, (3,3), padding="same", activation="relu", kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3,3), padding="same", activation="relu", kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Dropout(0.45)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inputs, outputs)

cnn = make_cnn(input_shape, num_classes)
cnn.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
            loss="categorical_crossentropy",
            metrics=["accuracy"])
cnn.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 306,602 (1.17 MB)

 Trainable params: 305,706 (1.17 MB)

 Non-trainable params: 896 (3.50 KB)

### 2.3 Treinamento com Data Augmentation e Callbacks

Nesta etapa, aplicamos **data augmentation** para aumentar a robustez do modelo:  
- rotações leves (até 5°),  
- flips horizontais,  
- deslocamentos de até 10% em largura/altura.  

Isso ajuda a CNN a **generalizar melhor** evitando overfitting.  

O treinamento utiliza **callbacks**:  
- *EarlyStopping*: interrompe se não houver melhora na validação, restaurando os melhores pesos,  
- *ReduceLROnPlateau*: reduz a taxa de aprendizado quando o progresso estagna,  
- *ModelCheckpoint*: salva o melhor modelo durante o treino.  

A rede é treinada por até **50 épocas**, mas pode parar antes se a convergência estabilizar.

In [12]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=5, horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1)
train_gen = datagen.flow(x_train, y_train, batch_size=64)

callbacks=[
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
    tf.keras.callbacks.ModelCheckpoint("cnn_cifar10_best.keras", save_best_only=True)
]

hist = cnn.fit(train_gen, epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)

Epoch 1/50


/Users/luccahiratsuca/Github/teste/Ponderada-CNN-s/env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 63s 78ms/step - accuracy: 0.4240 - loss: 1.6280 - val_accuracy: 0.5444 - val_loss: 1.3050 - learning_rate: 0.0010
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 62s 80ms/step - accuracy: 0.5719 - loss: 1.2567 - val_accuracy: 0.6145 - val_loss: 1.1359 - learning_rate: 0.0010
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 63s 80ms/step - accuracy: 0.6323 - loss: 1.1032 - val_accuracy: 0.6721 - val_loss: 0.9864 - learning_rate: 0.0010
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 62s 80ms/step - accuracy: 0.6723 - loss: 1.0109 - val_accuracy: 0.6584 - val_loss: 1.0921 - learning_rate: 0.0010
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 65s 83ms/step - accuracy: 0.7016 - loss: 0.9501 - val_accuracy: 0.6785 - val_loss: 1.0461 - learning_rate: 0.0010
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 62s 80ms/step - accuracy: 0.7193 - loss: 0.9132 - val_accuracy: 0.7238 - val_loss: 0.8969 - learning_rate: 0.0010
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 64s 81ms/step - accuracy: 0.7355 - loss: 0.8793

In [13]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

test_loss, test_acc = cnn.evaluate(x_test, y_test, verbose=0)
print("Test accuracy:", test_acc)

y_pred = np.argmax(cnn.predict(x_test, verbose=0), axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)
print(classification_report(y_true, y_pred))

Test accuracy: 0.866599977016449
Confusion matrix:
 [[854  13  23   8   9   0  13   9  55  16]
 [  2 968   1   0   0   0   2   0   4  23]
 [ 40   2 800  13  35  21  67  14   3   5]
 [ 11   6  29 672  53  75 107  23  15   9]
 [  5   1  23   6 875   4  58  27   1   0]
 [  2   3  29  99  39 732  53  36   1   6]
 [  1   2   7   7   4   1 973   2   1   2]
 [  7   1   6  13  28   9  16 917   1   2]
 [ 16   6   2   2   4   0   5   0 954  11]
 [  8  45   2   1   1   1   6   3  12 921]]
              precision    recall  f1-score   support

           0       0.90      0.85      0.88      1000
           1       0.92      0.97      0.95      1000
           2       0.87      0.80      0.83      1000
           3       0.82      0.67      0.74      1000
           4       0.83      0.88      0.85      1000
           5       0.87      0.73      0.79      1000
           6       0.75      0.97      0.85      1000
           7       0.89      0.92      0.90      1000
           8       0.91      0

No experimento com CIFAR-10, a CNN alcançou **acurácia de ~86,7%**, com desempenho elevado em classes mais fáceis (como `1`, `8` e `9`, todas com precisão/recall >0.90) e maior dificuldade em classes mais ambíguas (`3`, `5` e `6`, com recall entre 0.67–0.73). Esses resultados confirmam a robustez da arquitetura proposta, mas também ressaltam um ponto crítico: **classes minoritárias ou mais difíceis tendem a sofrer em recall**. Essa observação é diretamente análoga ao problema de fraude, onde os casos positivos são raros e o foco deve estar em **maximizar recall sem sacrificar demais a precisão**. Assim, a análise de CIFAR-10 reforça a importância de técnicas como **ajuste de limiar, métricas baseadas em PR AUC e tratamento do desbalanceamento** para melhorar a cobertura de eventos raros, como fraudes.

In [14]:
### 2.4 Salvando artefatos CNN
import json, pandas as pd, matplotlib.pyplot as plt
cnn.save("cnn_cifar10_final.keras")

with open("cnn_history.json","w") as f: json.dump(hist.history,f,indent=2)
with open("cnn_test_metrics.json","w") as f: json.dump({"test_loss":float(test_loss),"test_acc":float(test_acc)},f,indent=2)
pd.DataFrame(cm).to_csv("cnn_confusion_matrix.csv",index=False)
with open("cnn_classification_report.json","w") as f: json.dump(classification_report(y_true,y_pred,output_dict=True),f,indent=2)

plt.plot(hist.history["accuracy"]); plt.plot(hist.history["val_accuracy"]); plt.title("Acurácia"); plt.legend(["Treino","Val"]); plt.savefig("cnn_curva_acuracia.png"); plt.close()
plt.plot(hist.history["loss"]); plt.plot(hist.history["val_loss"]); plt.title("Perda"); plt.legend(["Treino","Val"]); plt.savefig("cnn_curva_perda.png"); plt.close()

print("✅ Artefatos CNN salvos!")

✅ Artefatos CNN salvos!


## 5) Relação com Detecção de Fraudes (tabular)

Embora tenhamos mostrado a engenharia de uma CNN para imagens, o problema de fraude lida majoritariamente com dados tabulares.  
Boas práticas indicam:
- **MLP (rede densa)** com padronização e regularização é mais apropriada que CNNs.
- **Seleção de limiar** é crítica: calibramos para **alta precisão** (minimizar falsos positivos) com o maior recall possível.
- Curvas **PR** são mais informativas que ROC em desbalanceamento extremo.

A seguir, treinamos um modelo inicial de fraude (pipeline `StandardScaler + MLP(64,32)`), calibramos limiar e avaliamos métricas.

In [15]:
import numpy as np, json, joblib, matplotlib.pyplot as plt, pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    confusion_matrix, roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve,
    precision_score, recall_score, f1_score
)

RANDOM_STATE = 42

X, y = make_classification(n_samples=30000, n_features=30, n_informative=12, n_redundant=6,
                           n_classes=2, weights=[0.99,0.01], flip_y=0.001,
                           class_sep=1.2, random_state=RANDOM_STATE)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=RANDOM_STATE)

pipe = Pipeline([("scaler", StandardScaler()),
                 ("mlp", MLPClassifier(hidden_layer_sizes=(64,32), activation="relu", solver="adam",
                                       alpha=1e-4, batch_size=256, learning_rate_init=1e-3,
                                       max_iter=200, early_stopping=True, validation_fraction=0.1,
                                       n_iter_no_change=10, random_state=RANDOM_STATE))])
pipe.fit(X_train,y_train)

proba = pipe.predict_proba(X_test)[:,1]
prec, rec, thr = precision_recall_curve(y_test, proba)
best_thr, best_rec = 0.5, -1
for p,r,t in zip(prec[:-1],rec[:-1],thr):
    if p>=0.90 and r>best_rec:
        best_thr, best_rec = float(t), r
if best_rec<0:
    from sklearn.metrics import f1_score
    f1s=[f1_score(y_test,(proba>=t).astype(int)) for t in thr]
    best_thr=float(thr[int(np.argmax(f1s))])

y_pred=(proba>=best_thr).astype(int)
metrics={"threshold_selected":best_thr,
         "test_roc_auc":roc_auc_score(y_test,proba),
         "test_pr_auc":average_precision_score(y_test,proba),
         "test_precision_at_tuned":precision_score(y_test,y_pred,zero_division=0),
         "test_recall_at_tuned":recall_score(y_test,y_pred,zero_division=0),
         "test_f1_at_tuned":f1_score(y_test,y_pred,zero_division=0),
         "confusion_matrix":confusion_matrix(y_test,y_pred).tolist()}

print(json.dumps(metrics,indent=2))

# salvar artefatos
joblib.dump({"pipeline":pipe,"threshold":best_thr},"fraud_mlp_sklearn.joblib")
with open("sklearn_metrics.json","w") as f: json.dump(metrics,f,indent=2)

fpr,tpr,_=roc_curve(y_test,proba); plt.plot(fpr,tpr); plt.savefig("sklearn_roc_curve.png"); plt.close()
P,R,_=precision_recall_curve(y_test,proba); plt.plot(R,P); plt.savefig("sklearn_pr_curve.png"); plt.close()

print("✅ Artefatos fraude salvos!")

{
  "threshold_selected": 0.6273491336540353,
  "test_roc_auc": 0.9329387136360355,
  "test_pr_auc": 0.7191904637712943,
  "test_precision_at_tuned": 0.918918918918919,
  "test_recall_at_tuned": 0.5396825396825397,
  "test_f1_at_tuned": 0.68,
  "confusion_matrix": [
    [
      5934,
      3
    ],
    [
      29,
      34
    ]
  ]
}
✅ Artefatos fraude salvos!


## 6) Hiperparâmetros

A escolha dos hiperparâmetros foi guiada por literatura especializada e práticas consolidadas em aprendizado profundo.  

**CNN (CIFAR-10):**  
- **Arquitetura:** blocos Conv2D + BatchNorm + ReLU, inspirados em VGG.  
- **Dropout:** valores progressivos [0.25, 0.35, 0.45, 0.5] para reduzir overfitting em diferentes profundidades.  
- **Regularização L2:** λ = 1e-4 para penalizar pesos excessivos.  
- **Pooling:** GlobalAveragePooling substitui Flatten, reduzindo parâmetros e overfitting.  
- **Camada densa final:** 128 neurônios com ReLU.  
- **Otimizador:** Adam (lr=1e-3), equilibrando robustez e velocidade de convergência.  
- **Callbacks:** EarlyStopping (paciente a 10 épocas), ReduceLROnPlateau (fator 0.5) e ModelCheckpoint (melhor modelo salvo).

**MLP (Fraude Tabular):**  
- **Pré-processamento:** StandardScaler para normalizar variáveis contínuas.  
- **Camadas ocultas:** (64, 32) neurônios com ReLU, suficiente para capturar interações sem overfitting.  
- **Regularização L2:** λ = 1e-4 para aumentar generalização.  
- **Batch size:** 256, equilibrando eficiência e estabilidade do gradiente.  
- **Otimizador:** Adam (lr=1e-3).  
- **Treinamento:** EarlyStopping (max_iter=200, validação interna 10%).  
- **Seleção de limiar:** calibrado para **precisão ≥ 0.90**, garantindo baixa taxa de falsos positivos e buscando o maior recall possível.  


## Conclusão

- A arquitetura **CNN** para CIFAR-10 confirmou boas práticas de redes convolucionais: uso de BatchNorm, Dropout progressivo, regularização L2 e GlobalAveragePooling resultaram em uma **acurácia de teste de ~86,7%**, consistente com benchmarks.  
- Para o domínio de **fraude tabular**, aplicamos um **MLP denso calibrado por limiar**, estratégia mais adequada que CNNs para este tipo de dado. O modelo alcançou **ROC AUC de 0.933, PR AUC de 0.719, precisão ~0.919 e recall ~0.540**, um **trade-off equilibrado** entre minimização de falsos positivos e cobertura de fraudes.  
- Esses resultados reforçam a importância da **seleção de limiar** e do uso de **curvas PR** em cenários altamente desbalanceados.  
- **Próximos passos** incluem:
  - **Engenharia de atributos** baseada em domínio (histórico de transações, dispositivos, geolocalização).  
  - **Validação temporal**, mais realista para fraudes, respeitando ordem cronológica.  
  - **Modelos tabulares avançados** (XGBoost, LightGBM, TabNet, FT-Transformer) como baseline competitivo.  
  - **Calibração probabilística** (Platt scaling, isotonic regression) para melhorar confiabilidade das probabilidades previstas.  